# SDM + RT
Single-diode model with the RTFrance dataset.

In [11]:
# using Plots
using PSO
using CUDA, Random

In [12]:
const data_dir = "../data/"
include("utils.jl")

benchmark (generic function with 1 method)

In [13]:
# set random seed: for reproduction
Random.seed!(CURAND.default_rng(), 123)

## Read data and set up search range

In [14]:
# select data type to be used on GPU: Float64 or Float32
# Float32 tends to be a little faster, but with lower precision
const TF = Float64

Float64

In [15]:
const T = TF(33 + 273.15)  # temperature in Kalvin of the RTC France case
const data = read_data(joinpath(data_dir, "RTFrance.csv"), TF);

In [16]:
const x_min = TF[0, 0, 1, 0, 0];
const x_max = TF[1, 1, 2, 0.5, 100];

## PSO parameters

In [17]:
const w = TF(0.5)
const c1 = TF(2.5)
const c2 = TF(1.6)
const np = 3200
const niter = 200
const min_delta=TF(1e-10)
const patience=20
const threads=320  # threads per block in CUDA
;

## Run PSO
SomceSS- Known best minimum in the literature: 2.52782e-05
- Note that the SSE (sum of square error) is minimized, which is equivalent to the minimization of RMSE.

In [18]:
# the time here may be much longer due to the JIT compile of Julia at the first fun of a function, but 
# much faster in subsequent runs.
@time pso(data, T, :SDM, x_min, x_max; niter, np, w, c1, c2, log=true, threads, min_delta, patience)

(threads, blocks) = (320, 10)
iter      sse       
1         6.61115e-02
2         1.53647e-02
3         1.17939e-02
4         5.04713e-03
5         5.04713e-03
6         3.45385e-03
7         7.39331e-04
8         6.65664e-04
9         5.06939e-04
10        3.37846e-04
11        3.35368e-04
12        2.16294e-04
13        2.16294e-04
14        1.99893e-04
15        1.99893e-04
16        1.74015e-04
17        1.73981e-04
18        1.39226e-04
19        1.22713e-04
20        1.18612e-04
21        1.14527e-04
22        1.14102e-04
23        1.06217e-04
24        9.98422e-05
25        9.01607e-05
26        8.39129e-05
27        8.01119e-05
28        7.02107e-05
29        6.70242e-05
30        6.44180e-05
31        6.18070e-05
32        5.86682e-05
33        5.54364e-05
34        5.19834e-05
35        5.16524e-05
36        5.04792e-05
37        5.03066e-05
38        4.96261e-05
39        4.65912e-05
40        4.55448e-05
41        4.06615e-05
42        3.83983e-05
43        3.39181e-05
44 

(sse = 2.527821774171829e-5, solution = [0.7607755457336648, 0.32301921022163566, 1.4811830947516174, 0.03637711347267447, 53.718296454145204], stopped_iter = 98)

## Measure statistics
Run PSO multiple times and record the results. Analyze the results.

In [19]:
f = () -> pso(data, T, :SDM, x_min, x_max; niter, np, w, c1, c2, log=false, threads, min_delta, patience)
measure_RMSE_stats(f, size(data, 2); n_trials=30)

Dict{Symbol, Float64} with 4 entries:
  :max  => 0.000986022
  :std  => 7.07982e-13
  :mean => 0.000986022
  :min  => 0.000986022

## Benchmark the running time
Run the PSO multiple times and measure the average running time. [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) is a library devoted to benchmarking in the `benchmark` function defined in `utils.jl`.

Due to GPU characteristics as well as early stopping, the variance among run time can be fairly high.

In [20]:
benchmark(f; n_trials=100)

BenchmarkTools.Trial: 
  memory estimate:  764.94 KiB
  allocs estimate:  16630
  --------------
  minimum time:     12.961 ms (0.00% GC)
  median time:      366.476 ms (0.00% GC)
  mean time:        235.497 ms (0.69% GC)
  maximum time:     490.802 ms (0.00% GC)
  --------------
  samples:          100
  evals/sample:     1